In [1]:
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
data_file = './/Amazon_Unlocked_Mobile.csv'
n = 413000  
s = 40000 
skip = sorted(random.sample(range(1,n),n-s))


df = pd.read_csv( data_file, delimiter = ",", skiprows = skip)

In [3]:
df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
2,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,Excelente,0.0
3,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,solid phone,0.0
4,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,4,"perfect, practical, easy to handle, good resol...",0.0
...,...,...,...,...,...,...
40835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
40836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
40837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
40838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [4]:
df.isnull().sum()

Product Name       0
Brand Name      6227
Price            553
Rating             0
Reviews           12
Review Votes    1217
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

Product Name    0
Brand Name      0
Price           0
Rating          0
Reviews         0
Review Votes    0
dtype: int64

In [7]:
df
#결측치들을 제거하니 41만개의 데이터에서 33만개의 데이터로 축소되었음
#아마 하나의 row에 결측치가 한개라도 존재하면 해당 row를 제거하는 방법을 수행했을것으로 예상 

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,2,The charging port was loose. I got that solder...,0.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I originally was using the Samsung S2 Galaxy f...,0.0
2,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,Excelente,0.0
3,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,5,solid phone,0.0
4,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,4,"perfect, practical, easy to handle, good resol...",0.0
...,...,...,...,...,...,...
40835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
40836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
40837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
40838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


### Rating Mapping

In [8]:
Review_mapping =  {0: '0',1 : '0', 2 :'0',3 : '0', 4 : '1', 5 : '1'}
Review_mapping
#0~3점까지는 0으로 4~5점까지는 1로 맵핑 

{0: '0', 1: '0', 2: '0', 3: '0', 4: '1', 5: '1'}

In [9]:
Rating = lambda x: Review_mapping.get(x,x)
df['Rating']=df.Rating.map(Rating)

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
df['Rating'].unique()

array(['0', '1'], dtype=object)

In [11]:
df
#rating이 잘변환된것을 확인할 수 있다.

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,0,The charging port was loose. I got that solder...,0.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,1,I originally was using the Samsung S2 Galaxy f...,0.0
2,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,1,Excelente,0.0
3,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,1,solid phone,0.0
4,"""Nokia Asha 302 Unlocked GSM Phone with 3.2MP ...",Nokia,299.00,1,"perfect, practical, easy to handle, good resol...",0.0
...,...,...,...,...,...,...
40835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,another great deal great price,0.0
40836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,Ok,0.0
40837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,1,Passes every drop test onto porcelain tile!,0.0
40838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,0,I returned it because it did not meet my needs...,0.0


### 텍스트 데이터 분리

In [12]:
df.columns

Index(['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews',
       'Review Votes'],
      dtype='object')

In [13]:
a = df.iloc[:,3]
print(a)
# a는 평점
a.astype('int64')
#맵핑을 한 rating이 object로 되어있어 int로 변환을 하였음 

0        0
1        1
2        1
3        1
4        1
        ..
40835    1
40836    0
40837    1
40838    0
40839    1
Name: Rating, Length: 33211, dtype: object


0        0
1        1
2        1
3        1
4        1
        ..
40835    1
40836    0
40837    1
40838    0
40839    1
Name: Rating, Length: 33211, dtype: int64

In [14]:
b = df.iloc[:,4]
# b는 리뷰데이터(텍스트)

In [15]:
doc = pd.concat([a,b], axis = 1)
#기존의 데이터에서 rating과 review로 구성된 새로운 데이터셋을 구성 

In [16]:
def review_to_wordlist(review, remove_stopwords = True):

    
    review_text = re.sub('[^a-zA-Z]'," ", review)
    #review_text에 영어만 넣기 
    
    words = review_text.lower().split()
    #소문자로 바꿔주고 그것들을 분리해준다 
    
    if remove_stopwords:
        stops = set(stopwords.words('english'))
        words = [w for w in words if not w in stops]
        #stops에 영어의 불용어를 넣어줌 
        #words는 소문자로 변환되고 띄어져있는 것이며 
        #stops에 있는 불용어를 제외하고 넣어줌 
        
    b = []
    stemmer = english_stemmer
    for word in words:
        b.append(stemmer.stem(word))
        #words에 전처리된것들의 어간들만 추출 
        
    return(b)


In [17]:
clean_reviews = []
for review in doc['Reviews']:
    clean_reviews.append( " ".join(review_to_wordlist(review)))

In [18]:
len(clean_reviews)
#각 row마다 텍스트 전처리가 잘된것을 확인 

33211

In [19]:
c = clean_reviews
type(c)

list

In [20]:
clean_reviews = pd.DataFrame(clean_reviews)

In [21]:
clean_reviews.columns = ['Reviews']
clean_reviews

,Reviews
0,charg port loos got solder need new batteri we...
1,origin use samsung galaxi sprint want return b...
2,excelent
3,solid phone
4,perfect practic easi handl good resolut well pack
...,...
33206,anoth great deal great price
33207,ok
33208,pass everi drop test onto porcelain tile
33209,return meet need seem good select other


In [22]:
df = pd.concat([clean_reviews, doc['Rating']], axis = 1)
df.isnull().sum()
#무작정 concat을 사용했기때문에 결측값이 발생 

Reviews    5715
Rating     5715
dtype: int64

In [23]:
clean_reviews.reset_index(drop=True , inplace=True)
clean_reviews

,Reviews
0,charg port loos got solder need new batteri we...
1,origin use samsung galaxi sprint want return b...
2,excelent
3,solid phone
4,perfect practic easi handl good resolut well pack
...,...
33206,anoth great deal great price
33207,ok
33208,pass everi drop test onto porcelain tile
33209,return meet need seem good select other


In [24]:
b = doc['Rating']

In [25]:
b.reset_index(drop = True, inplace = True)
b

0        0
1        1
2        1
3        1
4        1
        ..
33206    1
33207    0
33208    1
33209    0
33210    1
Name: Rating, Length: 33211, dtype: object

In [26]:
Data = pd.concat([clean_reviews, doc['Rating']], axis = 1)
Data.isnull().sum()
#인덱스를 초기화해주지않아서 문제가 발생을 하였다

Reviews    0
Rating     0
dtype: int64

In [27]:
Data

,Reviews,Rating
0,charg port loos got solder need new batteri we...,0
1,origin use samsung galaxi sprint want return b...,1
2,excelent,1
3,solid phone,1
4,perfect practic easi handl good resolut well pack,1
...,...,...
33206,anoth great deal great price,1
33207,ok,0
33208,pass everi drop test onto porcelain tile,1
33209,return meet need seem good select other,0


In [28]:
train_docs, test_docs = train_test_split(Data, test_size = 0.3)

# ================ 5/1

### Tf-Idf

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk.corpus import words

vectorizer = CountVectorizer(analyzer = 'word', 
                             lowercase = True,
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = 'english',
                             min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3),
                             vocabulary = set(words.words()), # nltk의 words를 사용하거나 문서 자체의 사전을 만들거나 선택한다. 
                             max_features = 20000
                            )

pipeline = Pipeline([
    ('vect', vectorizer),
    ('tfidf', TfidfTransformer(smooth_idf = False))
])

X_train_tf_idf_vector = pipeline.fit_transform(X_train)

X_test_tf_idf_vector = pipeline.fit_transform(X_test)

### ====================== 사용 x

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
NGRAM_RANGE = (1,2)
TOP_K = 25000
TOKEN_MODE = 'word'
MIN_DOC_FREQ = 2

In [41]:
def ngram_vectorize(texts, labels):
    kwargs = {
        'ngram_range' : NGRAM_RANGE,
        'dtype' : 'int32',
        'strip_accents' : 'unicode',
        'decode_error' : 'replace',
        'analyzer' : TOKEN_MODE,
        'min_df' : MIN_DOC_FREQ
    }
    
    tfidf_vectorizer = TfidfVectorizer(**kwargs)
    transformed_texts = tfidf_vectorizer.fit_transform(texts)
    
    selector = SelectKBest(f_classif, k = min(TOP_K, transformed_texts.shape[1]))
    selector.fit(transformed_texts, labels)
    
    transformed_texts = selector.transform(transformed_texts).astype('float32')
    return transformed_texts

In [42]:
vect_train_data = ngram_vectorize(train_docs['Reviews'], train_docs['Rating'])
vect_train_data

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


<23247x25000 sparse matrix of type '<class 'numpy.float32'>'
	with 472673 stored elements in Compressed Sparse Row format>

In [43]:
vect_test_data = ngram_vectorize(test_docs['Reviews'], test_docs['Rating'])
vect_test_data

/Users/Moon/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:1616: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. int32 'dtype' will be converted to np.float64.
  UserWarning)


<9964x25000 sparse matrix of type '<class 'numpy.float32'>'
	with 247897 stored elements in Compressed Sparse Row format>

In [141]:
rf_X_train = vect_train_data.toarray()
rf_y_train = (np.array(train_docs['Rating']))
rf_X_test = vect_test_data.toarray()
rf_y_test = (np.array(test_docs['Rating']))

### Random Forest

In [142]:
from sklearn.ensemble import RandomForestClassifier

In [143]:
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2020)
forest
#100회의 학습을 수행하며 나머지 파라미터들은 default값으로 지정
#데이터가 너무 커서 최적의 파라미터들을 찾기보다는 default로 두는 것이 가장 좋은 선택이라는 판단 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=2020, verbose=0,
                       warm_start=False)

In [144]:
forest = forest.fit(rf_X_train, rf_y_train)

In [145]:
result = forest.predict(rf_X_test)

In [152]:
rf_y_test = rf_y_test.astype('int64')

In [153]:
rf_y_test

array([1, 1, 1, ..., 1, 0, 1])

In [157]:
result = result.astype('int64')

In [161]:
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [162]:
get_clf_eval(rf_y_test, result)

오차행렬:
 [[  87 2983]
 [ 104 6790]]

정확도: 0.6902
정밀도: 0.6948
재현율: 0.9849
F1: 0.8148
AUC: 0.5066


### XGBOOST

In [163]:
import xgboost as xgb 
from xgboost import plot_importance 
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from xgboost import XGBClassifier

In [164]:
dtrain = xgb.DMatrix(data = rf_X_train, label = rf_y_train)
dtest = xgb.DMatrix(data= rf_X_test, label = rf_y_test)

#numpy형태의 학습 데이터셋과 테스트 데이터셋을 DMatirx로 변환

In [168]:
params = {'max_depth' : 4,
          'eta' : 0.1,
          'objective' : 'binary:logistic',
          'eval_metric' : 'logloss',
          'early_stoppings' : 30}

num_rounds = 100
#eta : 학습률 
#손실함수 평가지표 : logloss
#파라미터를 

In [169]:
wlist = [(dtrain, 'train'), (dtest, 'eval')]
xgb_model = xgb.train(params = params, 
                      dtrain = dtrain, 
                      num_boost_round = num_rounds,
                      evals = wlist)
#학습을 100회부터 400회까지 돌려본결과 100회부터는 학습에 크게 영향을 주지 않는다


[22:06:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { early_stoppings } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-logloss:0.66528	eval-logloss:0.68477
[1]	train-logloss:0.64231	eval-logloss:0.67783
[2]	train-logloss:0.62337	eval-logloss:0.66919
[3]	train-logloss:0.60700	eval-logloss:0.66481
[4]	train-logloss:0.59333	eval-logloss:0.65739
[5]	train-logloss:0.58099	eval-logloss:0.65449
[6]	train-logloss:0.57050	eval-logloss:0.64965
[7]	train-logloss:0.56000	eval-logloss:0.64968
[8]	train-logloss:0.55164	eval-logloss:0.64390
[9]	train-logloss:0.54307	eval-logloss:0.64386
[10]	train-logloss:0.53516	eval-logloss:0.64499
[11]	train-logloss:0.52879	eval-logloss:0.64046
[12]	train-logloss:0.52207	eval-logloss:0.63750
[13]	train-logloss:0.

In [170]:
pred_probs = xgb_model.predict(dtest)
#예측값을 확률로 출력 
print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10], 3))

predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.62 0.62 0.62 0.62 0.62 0.62 0.62 0.62 0.62 0.62]


In [171]:
preds = [1 if x > 0.5 else 0 for x in pred_probs]

In [173]:
#Confusion Matrix, Accuracy, precision, recall, F1, AUC
def get_clf_eval(y_test, preds):
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    F1 = f1_score(y_test, preds)
    AUC = roc_auc_score(y_test, preds)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))

In [174]:
get_clf_eval(rf_y_test, preds)
#재현율에서는 아주 높은 값을 보여주고 있지만 나머지값들에 대해서는 좋지 못한 결과를 보여준다
#또한 AUC가 0.5정도면 좋지 못한 모델이라는 것을 보여주며 추후 수정이 필요함 

오차행렬:
 [[  28 3042]
 [  47 6847]]

정확도: 0.6900
정밀도: 0.6924
재현율: 0.9932
F1: 0.8159
AUC: 0.5012


### LSTM

In [175]:
X_train = rf_X_train
y_train = rf_y_train 
X_test = rf_X_test
y_test = rf_y_test

In [176]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [177]:
embed_size = 128
max_features = 25000


model = Sequential()
model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

#6개의 layer로 구성되어 있으며 성능을 좋게 내기 위해 학습 모델 코드를 참조하였음 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         3200000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 64)          41216     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1300      
_________________________________________________________________
dropout_1 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 21        
Total params: 3,242,537
Trainable param

In [179]:
batch_size = 128
epochs = 4
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Train on 18597 samples, validate on 4650 samples
Epoch 1/4
18597/18597 [==============================] - 9438s 507ms/step - loss: 4.7128 - accuracy: 0.6926 - val_loss: 4.8535 - val_accuracy: 0.6817
Epoch 2/4
18597/18597 [==============================] - 9297s 500ms/step - loss: 4.7128 - accuracy: 0.6926 - val_loss: 4.8535 - val_accuracy: 0.6817
Epoch 3/4
18597/18597 [==============================] - 9208s 495ms/step - loss: 4.7128 - accuracy: 0.6926 - val_loss: 4.8535 - val_accuracy: 0.6817
Epoch 4/4
18597/18597 [==============================] - 9415s 506ms/step - loss: 4.7128 - accuracy: 0.6926 - val_loss: 4.8535 - val_accuracy: 0.6817


In [ ]:
prediction = model.predict(X_test)
y_pred = (prediction > 0.5)

In [ ]:
get_clf_eval(y_test, y_preds)